# CDA Python: Features & Examples
---

The following examples illustrate some ```CDA Python``` features while providing insights into the underlying data structure (**Getting started**). To demonstrate those features, we provide a few relevant text queries along with step-by-step explanations on how to translate those into the ```CDA Python``` queries (**Example queries**). Finally, there are a few additional queries intended for the test users to play around with and send feedback to the CDA team (**Test queries**).

In [1]:
from cdapython import Q, columns, unique_terms

## Getting started

Print out the list of available fields with ```columns()```:

In [5]:
columns()

SELECT field_path FROM `gdc-bq-sample.cda_mvp.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS` WHERE table_name = 'v3'


['days_to_birth',
 'race',
 'sex',
 'ethnicity',
 'id',
 'ResearchSubject',
 'ResearchSubject.Diagnosis',
 'ResearchSubject.Diagnosis.morphology',
 'ResearchSubject.Diagnosis.tumor_stage',
 'ResearchSubject.Diagnosis.tumor_grade',
 'ResearchSubject.Diagnosis.Treatment',
 'ResearchSubject.Diagnosis.Treatment.type',
 'ResearchSubject.Diagnosis.Treatment.outcome',
 'ResearchSubject.Diagnosis.id',
 'ResearchSubject.Diagnosis.primary_diagnosis',
 'ResearchSubject.Diagnosis.age_at_diagnosis',
 'ResearchSubject.Specimen',
 'ResearchSubject.Specimen.File',
 'ResearchSubject.Specimen.File.label',
 'ResearchSubject.Specimen.File.associated_project',
 'ResearchSubject.Specimen.File.drs_uri',
 'ResearchSubject.Specimen.File.identifier',
 'ResearchSubject.Specimen.File.identifier.system',
 'ResearchSubject.Specimen.File.identifier.value',
 'ResearchSubject.Specimen.File.data_category',
 'ResearchSubject.Specimen.File.byte_size',
 'ResearchSubject.Specimen.File.type',
 'ResearchSubject.Specimen.File

All of the above fields are what describes the highest entity in the data structure hierarchy – ```Patient``` entity. The first five fields represent ```Patient``` demographic information, while the ```ResearchSubject``` entity contains details that we are used to seeing within the nodes' ```Case``` record.

One of the contributions of the CDA is aggregated ```ResearchSubject``` information. This means that all ```ResearchSubject``` records coming from the same subject are now gathered under the Patient entity. As we know, certain specimens are studied in multiple projects (being part of a single data node or multiple nodes) as different ```ResearchSubject``` entries. Those ```ResearchSubject``` entries are collected as a list under the ```ResearchSubject``` entity. One example of this is the patient record with ```id = TCGA-E2-A10A``` which contains two ```ResearchSubject``` entries, one from GDC and the other from PDC.

Note that the ```ResearchSubject``` entity is a list of records, as many other entities above are. **There are certain considerations that should be made when creating the queries by using the fields that come from lists, but more about that will follow in examples below**.

The names in the list may look familiar to you, but they may have been renamed or restructured in the CDA. The field name mappings are described in the _CDA Schema Field Mapping_ document that is linked in the _Testing Guide_. A more direct way to explore and understand the fields is to use the ```unique_terms()``` function:

In [3]:
unique_terms("ResearchSubject.Specimen.source_material_type")

SELECT DISTINCT(_Specimen.source_material_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen ORDER BY _Specimen.source_material_type


['Additional - New Primary',
 'Additional Metastatic',
 'Blood Derived Cancer - Bone Marrow',
 'Blood Derived Cancer - Bone Marrow, Post-treatment',
 'Blood Derived Cancer - Peripheral Blood',
 'Blood Derived Cancer - Peripheral Blood, Post-treatment',
 'Blood Derived Normal',
 'Bone Marrow Normal',
 'Buccal Cell Normal',
 'Cell Lines',
 'Control Analyte',
 'DNA',
 'Expanded Next Generation Cancer Model',
 'FFPE Scrolls',
 'Fibroblasts from Bone Marrow Normal',
 'Granulocytes',
 'Lymphoid Normal',
 'Metastatic',
 'Mononuclear Cells from Bone Marrow Normal',
 'Neoplasms of Uncertain and Unknown Behavior',
 'Next Generation Cancer Model',
 'Normal',
 'Normal Adjacent Tissue',
 'Not Reported',
 'Post neo-adjuvant therapy',
 'Primary Blood Derived Cancer - Bone Marrow',
 'Primary Blood Derived Cancer - Peripheral Blood',
 'Primary Tumor',
 'Primary Xenograft Tissue',
 'Recurrent Blood Derived Cancer - Bone Marrow',
 'Recurrent Blood Derived Cancer - Peripheral Blood',
 'Recurrent Tumor',
 

Additionally, you can specify a particular data node by using the ```system``` argument:

In [4]:
unique_terms("ResearchSubject.Specimen.source_material_type", system="PDC")

SELECT DISTINCT(_Specimen.source_material_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen,UNNEST(_ResearchSubject.identifier) AS _identifier WHERE _identifier.system="PDC" ORDER BY _Specimen.source_material_type


['Cell Lines',
 'Normal',
 'Normal Adjacent Tissue',
 'Not Reported',
 'Primary Tumor',
 'Solid Tissue Normal',
 'Tumor',
 'Xenograft',
 'Xenograft Tissue']

Now, let's dive into the querying!

We can start by getting the record for ```id = TCGA-E2-A10A``` that we mentioned earlier:

In [8]:
q = Q('id = "TCGA-E2-A10A"') # note the double quotes for the string value

r = q.run()

print(r)


Query: SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3 WHERE (v3.id = 'TCGA-E2-A10A')
Offset: 0
Count: 1
Total Row Count: 1
More pages: False



We see that we've got a single patient record as a result, which is what we expect.

Let's see how the result looks like:

In [ ]:
r[0]

The record is pretty large, so we'll print out ```identifier``` values for each ```ResearchSubject``` to confirm that we have one ```ResearchSubject``` that comes from GDC, and one that comes from PDC:

In [7]:
for research_subject in r[0]['ResearchSubject']:
    print(research_subject['identifier'])

[{'system': 'GDC', 'value': '4da7abaf-ac7a-41c0-8033-5780a398545c'}]
[{'system': 'PDC', 'value': '010df72d-63d9-11e8-bcf1-0a2705229b82'}]


The values represent ```ResearchSubject``` IDs and are equivalent to ```case_id``` values in data nodes.

## Example queries

Now that we can create a query with ```Q()``` function, let's see how we can combine multiple conditions.

There are three operators available:
* ```And()```
* ```Or()```
* ```From()```

The following examples show how those operators work in practice.

### Query 1

**Find data for subjects who were diagnosed after the age of 50 and who were investigated as part of the TCGA-OV project.**

In [8]:
q1 = Q('ResearchSubject.Diagnosis.age_at_diagnosis > 50*365')
q2 = Q('ResearchSubject.associated_project = "TCGA-OV"')

q = q1.And(q2)
r = q.run()

print(r)


Query: SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_Diagnosis.age_at_diagnosis > 50*365) AND (_ResearchSubject.associated_project = 'TCGA-OV'))
Offset: 0
Count: 461
Total Row Count: 461
More pages: False



### Query 2

**Find data for donors with melanoma (Nevi and Melanomas) diagnosis and who were diagnosed before the age of 30.**

In [9]:
q1 = Q('ResearchSubject.Specimen.primary_disease_type = "Nevi and Melanomas"')
q2 = Q('ResearchSubject.Diagnosis.age_at_diagnosis < 30*365')

q = q1.And(q2)
r = q.run()

print(r)


Query: SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_Specimen.primary_disease_type = 'Nevi and Melanomas') AND (_Diagnosis.age_at_diagnosis < 30*365))
Offset: 0
Count: 647
Total Row Count: 647
More pages: False



In addition, we can check how many records come from particular systems by adding one more condition to the query:

In [10]:
q1 = Q('ResearchSubject.Specimen.primary_disease_type = "Nevi and Melanomas"')
q2 = Q('ResearchSubject.Diagnosis.age_at_diagnosis < 30*365')
q3 = Q('ResearchSubject.Specimen.identifier.system = "GDC"')

q = q1.And(q2.And(q3))
r = q.run()

print(r)


Query: SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Specimen.identifier) AS _identifier WHERE ((_Specimen.primary_disease_type = 'Nevi and Melanomas') AND ((_Diagnosis.age_at_diagnosis < 30*365) AND (_identifier.system = 'GDC')))
Offset: 0
Count: 647
Total Row Count: 647
More pages: False



By comparing the ```Count``` value of the two results we can see that all the patients returned in the initial query are coming from the GDC.

To explore the results further, we can fetch the patient JSON objects by iterating through the results:

In [11]:
projects = set()

for patient in r:
    research_subjects = patient['ResearchSubject']
    for rs in research_subjects:
        projects.add(rs['associated_project'])

print(projects)

{'TCGA-UVM', 'TCGA-SKCM', 'FM-AD'}


The output shows the projects where _Nevi and Melanomas_ cases appear.

### Query 3

**Identify all samples that meet the following conditions:**

* **Sample is from primary tumor**
* **Disease is ovarian or breast cancer**
* **Subjects are females under the age of 60 years**

In [12]:
tumor_type = Q('ResearchSubject.Specimen.source_material_type = "Primary Tumor"')
disease1 = Q('ResearchSubject.primary_disease_site = "Ovary"')
disease2 = Q('ResearchSubject.primary_disease_site = "Breast"')
demographics1 = Q('sex = "female"')
demographics2 = Q('days_to_birth > -60*365') # note that days_to_birth is a negative value

q1 = tumor_type.And(demographics1.And(demographics2))
q2 = disease1.Or(disease2)
q = q1.And(q2)

r = q.run()
print(r)


Query: SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (((_Specimen.source_material_type = 'Primary Tumor') AND ((v3.sex = 'female') AND (v3.days_to_birth > -60*365))) AND ((_ResearchSubject.primary_disease_site = 'Ovary') OR (_ResearchSubject.primary_disease_site = 'Breast')))
Offset: 0
Count: 1000
Total Row Count: 27284
More pages: True



In this case, we have a result that contains more than 1000 records which is the default page size. To load the next 1000 records, we can use the ```next_page()``` method:

In [13]:
r2 = r.next_page()

In [14]:
print(r2)


Query: SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (((_Specimen.source_material_type = 'Primary Tumor') AND ((v3.sex = 'female') AND (v3.days_to_birth > -60*365))) AND ((_ResearchSubject.primary_disease_site = 'Ovary') OR (_ResearchSubject.primary_disease_site = 'Breast')))
Offset: 1000
Count: 1000
Total Row Count: 27284
More pages: True



Alternatively, we can use the ```offset``` argument to specify the record to start from:

```
...
r = q.run(offset=1000)
print(r)
```

### Query 4

**Find data for donors with "Ovarian Serous Cystadenocarcinoma" with proteomic and genomic data.**

**Note that disease type value denoting the same disease groups can be completely different within different systems. This is where CDA features come into play.** We first start by exploring the values available for this particular field in both systems.

In [15]:
unique_terms('ResearchSubject.primary_disease_type', system="GDC")

SELECT DISTINCT(_ResearchSubject.primary_disease_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.identifier) AS _identifier WHERE _identifier.system="GDC" ORDER BY _ResearchSubject.primary_disease_type


[None,
 'Acinar Cell Neoplasms',
 'Adenomas and Adenocarcinomas',
 'Adnexal and Skin Appendage Neoplasms',
 'Basal Cell Neoplasms',
 'Blood Vessel Tumors',
 'Chronic Myeloproliferative Disorders',
 'Complex Epithelial Neoplasms',
 'Complex Mixed and Stromal Neoplasms',
 'Cystic, Mucinous and Serous Neoplasms',
 'Ductal and Lobular Neoplasms',
 'Epithelial Neoplasms, NOS',
 'Fibroepithelial Neoplasms',
 'Fibromatous Neoplasms',
 'Germ Cell Neoplasms',
 'Giant Cell Tumors',
 'Gliomas',
 'Granular Cell Tumors and Alveolar Soft Part Sarcomas',
 'Hodgkin Lymphoma',
 'Immunoproliferative Diseases',
 'Leukemias, NOS',
 'Lipomatous Neoplasms',
 'Lymphatic Vessel Tumors',
 'Lymphoid Leukemias',
 'Malignant Lymphomas, NOS or Diffuse',
 'Mast Cell Tumors',
 'Mature B-Cell Lymphomas',
 'Mature T- and NK-Cell Lymphomas',
 'Meningiomas',
 'Mesonephromas',
 'Mesothelial Neoplasms',
 'Miscellaneous Bone Tumors',
 'Miscellaneous Tumors',
 'Mucoepidermoid Neoplasms',
 'Myelodysplastic Syndromes',
 'Myel

Since “Ovarian Serous Cystadenocarcinoma” doesn’t appear in GDC values we decide to look into the PDC:

In [16]:
unique_terms('ResearchSubject.primary_disease_type', system="PDC")

SELECT DISTINCT(_ResearchSubject.primary_disease_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.identifier) AS _identifier WHERE _identifier.system="PDC" ORDER BY _ResearchSubject.primary_disease_type


['Breast Invasive Carcinoma',
 'Chromophobe Renal Cell Carcinoma',
 'Clear Cell Renal Cell Carcinoma',
 'Colon Adenocarcinoma',
 'Early Onset Gastric Cancer',
 'Glioblastoma',
 'Head and Neck Squamous Cell Carcinoma',
 'Hepatocellular Carcinoma ',
 'Lung Adenocarcinoma',
 'Lung Squamous Cell Carcinoma',
 'Oral Squamous Cell Carcinoma',
 'Other',
 'Ovarian Serous Cystadenocarcinoma',
 'Pancreatic Ductal Adenocarcinoma',
 'Papillary Renal Cell Carcinoma',
 'Pediatric/AYA Brain Tumors',
 'Rectum Adenocarcinoma',
 'Uterine Corpus Endometrial Carcinoma']

After examining the output, we see that it does come from the PDC. Hence, if we could first identify the data that has research subjects found within the PDC that have this particular disease type, and then further narrow down the results to include only the portion of the data that is present in GDC, we could get the records that we are looking for.

In [17]:
q1 = Q('ResearchSubject.primary_disease_type = "Ovarian Serous Cystadenocarcinoma"')
q2 = Q('ResearchSubject.identifier.system = "PDC"')
q3 = Q('ResearchSubject.identifier.system = "GDC"')

q = q3.From(q1.And(q2))
r = q.run()

print(r)


Query: SELECT v3.* FROM (SELECT v3.* FROM gdc-bq-sample.cda_mvp.v3 AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.identifier) AS _identifier WHERE ((_ResearchSubject.primary_disease_type = 'Ovarian Serous Cystadenocarcinoma') AND (_identifier.system = 'PDC'))) AS v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.identifier) AS _identifier WHERE (_identifier.system = 'GDC')
Offset: 0
Count: 275
Total Row Count: 275
More pages: False



As you can see, this is achieved by utilizing ```From``` operator. The ```From``` operator allows us to create queries from results of other queries. This is particularly useful when working with conditions that involve a single field which can take multiple different values for different items in a list that is being part of, e.g. we need ```ResearchSubject.identifier.system``` to be both “PDC” and “GDC” for a single patient. In such cases, ```And``` operator can’t help because it will return those entries where the field takes both values, which is zero entries.

## Return all files associated with tumor specimens for a given patient
This is a Example for the static method **sql** this will let your write sql

In [26]:
for i in Q.sql('''
SELECT  SF.label, SF.drs_uri, SF.byte_size
FROM `gdc-bq-sample.cda_mvp.v3` AS gp,
UNNEST(gp.ResearchSubject) AS RS,
UNNEST(RS.Specimen) AS S,
UNNEST(S.File) AS SF
WHERE
    gp.id = "TCGA-E2-A10A"
    AND S.source_material_type = "Primary Tumor"
GROUP BY SF.id, SF.label, SF.drs_uri, SF.byte_size
LIMIT 100
'''):
    print(f'''
    label: {i["label"]}\n 
    drs_uri: {i["drs_uri"]}\n
    byte_size: {i["byte_size"]}\n
    ''')


    label: TCGA-E2-A10A-01Z-00-DX1.98B19EF1-0DAE-4DC6-8B0E-963CFABC6724.svs
 
    drs_uri: drs://dg.4DFC:99a00a9f-c4bf-49ca-9c3d-435f0a207644

    byte_size: 1341476123

    

    label: 0012f466-075a-4d47-b1d7-e8e63e8b9c99.vep.vcf.gz
 
    drs_uri: drs://dg.4DFC:0012f466-075a-4d47-b1d7-e8e63e8b9c99

    byte_size: 927367

    

    label: TCGA.BRCA.mutect.053f01ed-3154-4aea-9e7f-932c435034b3.DR-10.0.protected.maf.gz
 
    drs_uri: drs://dg.4DFC:053f01ed-3154-4aea-9e7f-932c435034b3

    byte_size: 1882061658

    

    label: 8a540cec-e283-4a26-868e-a07c64fe3efb.mirbase21.isoforms.quantification.txt
 
    drs_uri: drs://dg.4DFC:0b4727f6-8ed4-4ad6-a3fe-81a58dd7d033

    byte_size: 392954

    

    label: GHOUL_p_TCGASNP_b85and51R_N_GenomeWideSNP_6_C11_735316.nocnv_grch38.seg.v2.txt
 
    drs_uri: drs://dg.4DFC:15a9ba9e-95fa-4823-947f-f99b50456a40

    byte_size: 26302

    

    label: jhu-usc.edu_BRCA.HumanMethylation27.3.lvl-3.TCGA-E2-A10A-01A-21D-A112-05.gdc_hg38.txt
 
    drs_uri:

In [1]:
Q.statusbigquery()

NameError: name 'Q' is not defined

## Test queries

Now that we've successfully run and analyzed a few queries, here are a few additional ones you can try out on your own.

Solutions can be shared with the CDA team as indicated in the _Testing Guide_ document.

### Test Query 1

**Find data from TCGA-BRCA project, with donors over the age of 50 with Stage IIIC cancer.**

In [18]:
# Solution

# ...

# print(r)

### Test Query 2

**Find data from all patients who have been treated with "Radiation Therapy, NOS" and have both genomic and proteomic data.**

In [19]:
# Solution

# ...

# print(r)

### Test Query 3

**Find data from all subjects with lung adenocarcinomas that have both primary and recurrent tumors.**

In [20]:
# Solution

# ...

# print(r)